In [1]:
!pip install --quiet nltk

In [2]:
import re
import random
from datetime import datetime

# ----------------------------
# Patterns and responses (tunable)
# ----------------------------
PATTERNS = [
    (r".*\bmy name is\b\s*(.*)", ["Hello {1}! Nice to meet you. How are you today?"]),
    (r".*\bhelp\b.*", ["I can help. Ask about myself, how to use this notebook, or type 'quit' to leave."]),
    (r".*\byour name\b.*", ["I am a simple Colab chatbot. Call me 'bot'."]),
    (r"how are you\??", ["I'm a program — I don't have feelings, but I'm ready to assist!"]),
    (r".*\bsorry\b.*", ["No problem — it's okay."]),
    (r"i'm\s+(good|well|okay|ok)\b.*", ["Glad to hear that!"]),
    (r"^(hi|hey|hello)\b.*", ["Hello!", "Hey there!"]),
    (r".*\bwhat do you want\b.*", ["I want to help you build and test chat logic."]),
    (r".*\b(created|made)\b.*", ["I was created as a teaching example in Python."]),
    (r".*\b(location|city)\b.*", ["I exist inside this notebook — no physical city."]),
    (r".*\brain(ning)? ins\b\s*(.*)", ["I don't have live weather access here, but I hope it's nice in {2}."]),
    (r".*\b(health|wellness)\b.*", ["Health is important — remember to rest and hydrate."]),
    (r".*\b(sports|game|sport)\b.*", ["Do you have a favorite sport?"]),
    (r".*\b(quit|exit)\b.*", ["__EXIT__"]),  # sentinel for exit
    (r".*", ["That's interesting — tell me more."])
]

# ----------------------------
# Reflections dictionary
# ----------------------------
REFLECTIONS = {
    "i": "you",
    "me": "you",
    "my": "your",
    "am": "are",
    "you": "I",
    "your": "my",
    "yours": "mine",
    "i'm": "you are",
    "i'd": "you would",
    "i've": "you have",
    "i'll": "you will"
}

def apply_reflections(text):
    """Return a simple reflected version of the text using REFLECTIONS map."""
    # tokenization is intentionally simple to avoid extra dependencies
    tokens = re.findall(r"\w+|'t|'re|'ve|[-]|\S", text.lower())
    reflected_tokens = [REFLECTIONS.get(t, t) for t in tokens]
    return " ".join(reflected_tokens)

def normalize_input(user_text):
    """Normalize user text for pattern matching."""
    s = user_text.strip()
    s = re.sub(r"\s+", " ", s)
    s = s.strip(" .!?,;:")
    return s

def match_and_respond(user_text):
    """
    Match the user_text against PATTERNS and return a response string.
    If a pattern's response is '__EXIT__' the caller should handle exiting.
    """
    text = normalize_input(user_text)
    for pattern, responses in PATTERNS:
        m = re.match(pattern, text, flags=re.IGNORECASE)
        if m:
            resp_template = random.choice(responses)
            if resp_template == "__EXIT__":
                return "__EXIT__"
            # Replace {1}, {2} placeholders with reflected capture groups
            def repl(matchobj):
                idx = int(matchobj.group(1))
                if m.lastindex and idx <= m.lastindex:
                    captured = m.group(idx) or ""
                    return apply_reflections(captured)
                return ""
            response = re.sub(r"\{(\d+)\}", repl, resp_template)
            return response
    return "Hmm — I couldn't match that. Can you rephrase?"

# ----------------------------
# Notebook-friendly interfaces
# ----------------------------
_conversation_history = []  # list of tuples (timestamp, speaker, text)

def chat_once(user_text):
    """
    One-shot chat: send user_text, get bot response, and log both to history.
    Use this in Colab by calling chat_once("hello") repeatedly.
    Returns the bot reply (string). If '__EXIT__' is returned, calling cell should stop.
    """
    timestamp = datetime.now().isoformat(timespec='seconds')
    _conversation_history.append((timestamp, "user", user_text))
    response = match_and_respond(user_text)
    if response == "__EXIT__":
        exit_msg = "Bye for now — conversation ended."
        _conversation_history.append((datetime.now().isoformat(timespec='seconds'), "bot", exit_msg))
        return exit_msg
    _conversation_history.append((datetime.now().isoformat(timespec='seconds'), "bot", response))
    return response

def get_history():
    """Return the current conversation history as a list of (time, speaker, text)."""
    return list(_conversation_history)

def clear_history():
    """Clear the conversation history."""
    _conversation_history.clear()

# Optional blocking console-style function (works but will block the cell while waiting)
def chat_console():
    """A blocking console-style loop; works in Colab but blocks the executing cell."""
    print("Starting console chat. Type 'quit' or 'exit' to stop.")
    while True:
        try:
            user = input("You: ")
        except Exception:
            # In some Colab runs input() may raise; handle gracefully
            print("Input stopped or not available in this environment.")
            break
        if not user:
            print("Bot: Please type something.")
            continue
        resp = match_and_respond(user)
        if resp == "__EXIT__":
            print("Bot: Bye for now.")
            break
        print("Bot:", resp)

# Convenience: demonstration sequence (programmatic)
def demo_conversation():
    clear_history()
    demo_msgs = [
        "hi",
        "my name is Suhani",
        "how are you?",
        "it is raining in Mumbai",
        "who created you?",
        "quit"
    ]
    print("Demo conversation (programmatic):")
    for m in demo_msgs:
        print("You:", m)
        r = chat_once(m)
        print("Bot:", r)
    print("\nConversation history:")
    for row in get_history():
        print(row)

# If you want a quick demo, call demo_conversation() in a notebook cell.
print("Chatbot loaded. Use chat_once('your message') to get a reply, or call demo_conversation() to see an example.")


Chatbot loaded. Use chat_once('your message') to get a reply, or call demo_conversation() to see an example.


In [3]:
# Suppose you've run the chatbot cell above.
print(chat_once("hi"))
print(chat_once("my name is Suhani"))
print(chat_once("it is raining in Delhi"))
print(chat_once("quit"))   # ends the conversation (returns a goodbye message)
# View history:
get_history()


Hey there!
Hello suhani! Nice to meet you. How are you today?
That's interesting — tell me more.
Bye for now — conversation ended.


[('2025-11-21T14:54:45', 'user', 'hi'),
 ('2025-11-21T14:54:45', 'bot', 'Hey there!'),
 ('2025-11-21T14:54:45', 'user', 'my name is Suhani'),
 ('2025-11-21T14:54:45',
  'bot',
  'Hello suhani! Nice to meet you. How are you today?'),
 ('2025-11-21T14:54:45', 'user', 'it is raining in Delhi'),
 ('2025-11-21T14:54:45', 'bot', "That's interesting — tell me more."),
 ('2025-11-21T14:54:45', 'user', 'quit'),
 ('2025-11-21T14:54:45', 'bot', 'Bye for now — conversation ended.')]

In [4]:
demo_conversation()

Demo conversation (programmatic):
You: hi
Bot: Hello!
You: my name is Suhani
Bot: Hello suhani! Nice to meet you. How are you today?
You: how are you?
Bot: I'm a program — I don't have feelings, but I'm ready to assist!
You: it is raining in Mumbai
Bot: That's interesting — tell me more.
You: who created you?
Bot: I was created as a teaching example in Python.
You: quit
Bot: Bye for now — conversation ended.

Conversation history:
('2025-11-21T14:57:19', 'user', 'hi')
('2025-11-21T14:57:19', 'bot', 'Hello!')
('2025-11-21T14:57:19', 'user', 'my name is Suhani')
('2025-11-21T14:57:19', 'bot', 'Hello suhani! Nice to meet you. How are you today?')
('2025-11-21T14:57:19', 'user', 'how are you?')
('2025-11-21T14:57:19', 'bot', "I'm a program — I don't have feelings, but I'm ready to assist!")
('2025-11-21T14:57:19', 'user', 'it is raining in Mumbai')
('2025-11-21T14:57:19', 'bot', "That's interesting — tell me more.")
('2025-11-21T14:57:19', 'user', 'who created you?')
('2025-11-21T14:57:19'

In [ ]:
chat_console()

Starting console chat. Type 'quit' or 'exit' to stop.


In [ ]:
clear_history()